In [653]:
import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq

In [654]:
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [655]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

pio.templates.default = "plotly_white"

In [656]:
mmult = 4
mpad = mmult

dt = 0.1/mmult
T = 100./mmult
N = int(T/dt)


In [657]:
projnames = ['x', 'y', 'z']
npeaks = [0, 1]

In [658]:
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [659]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

In [660]:
# in a distant future all tables are merged into one
t = pd.DataFrame({
    'fx0':[1], 'fy0':[1], 'fz0':[2],
    'fx1':[2], 'fy1':[2], 'fz1':[1],
    'Ax0':[1], 'Ay0':[0.6], 'Az0':[0.5],
    'Ax1':[0], 'Ay1':[0], 'Az1':[0],
    'px0':[0], 'py0':[1/2], 'pz0':[0],
    'px1':[0], 'py1':[1/2], 'pz1':[0],
})

In [661]:
def build_orbit(row, tt, pos):
    for c in projnames:  # x, y, z
        pos[c] = 0
        for i in npeaks:  # 
            A, f, ph = row[f"A{c}{i}"], row[f"f{c}{i}"], row[f"p{c}{i}"]
            pos[c] += A*np.cos(2*np.pi*tt*f + np.pi*ph)
    return pos

In [662]:
def make_periodograms(x,y, mpad=mpad):
    R = np.sqrt(x**2 + y**2)
    PR = np.abs(fft(R, n=mpad*len(R)))
    Px = np.abs(fft(pos['x'], n=mpad*len(R)))
    Py = np.abs(fft(pos['y'], n=mpad*len(R)))
    return PR, Px, Py

In [663]:
mathdict = {'f':r"\nu", 'A': 'A', 'p': r'\varphi'}

def make_math_label(l):
    par, coord, i = l
    return '$' + mathdict[par] +'_' + coord + '^' + '{(' + i + ')}' +'$'

In [664]:
unicdict = {
    'f': "ν", 'A': 'A', 'p': 'φ'
}

def make_html_label(l):
    par, coord, i = l
    return unicdict[par] + \
        '<sub>' + coord + '</sub>' + \
        '<sup>' + i + '</sup>'

In [665]:
tt = np.arange(0, T, dt)
pos = pd.DataFrame(np.zeros(shape=(len(tt), 3)), columns=['x', 'y', 'z'])

In [666]:
ff = fftfreq(mpad*len(tt), dt)
R = np.sqrt(pos['x']**2 + pos['y']**2)

In [667]:
fig = make_subplots(rows=2, cols=3, 
                    specs=[[{}, {}, {}], [{"colspan": 3}, None, None]])

fig.add_trace(go.Scattergl(x=pos['x'], y=pos['y'],
                           line=dict(width=3, color='darkblue'),
                           showlegend=False),
              row=1, col=1)
fig.add_trace(go.Scattergl(x=pos['x'], y=pos['z'],
                           line=dict(width=3, color='darkblue'),
                           showlegend=False),
              row=1, col=2)
fig.add_trace(go.Scattergl(x=pos['y'], y=pos['z'],
                           line=dict(width=3, color='darkblue'),
                           showlegend=False),
              row=1, col=3)

fig.add_trace(go.Scattergl(x=ff, y=np.abs(fft(R)),name='PR'),
              row=2, col=1)
fig.add_trace(go.Scattergl(x=ff, y=np.abs(fft(pos['x'])), name='Px'),
              row=2, col=1)
fig.add_trace(go.Scattergl(x=ff, y=np.abs(fft(pos['y'])), name='Py'),
              row=2, col=1)

# equal scale
fig.update_layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    yaxis2=dict(scaleanchor="x2", scaleratio=1),
    yaxis3=dict(scaleanchor="x3", scaleratio=1),
    xaxis4=dict(range=[-5,5]),
    legend = dict(orientation='h')
)
fw = go.FigureWidget(fig)

In [668]:
widgs = dict()

def setup_sliders(*sliders, deschook=None, **kw):
    for sl in sliders:
        desc = make_html_label(sl) if deschook is None else deschook(make_html_label(sl))
        widgs[sl] = widgets.FloatSlider(description = desc,
                                        value=t.loc[0,sl], **kw)

In [669]:
setup_sliders('fx0', 'fx1', 'fy0', 'fy1', 
              min=0, max=10, step=0.001, readout_format='.3f')
setup_sliders('Ax0', 'Ax1', 'Ay0', 'Ay1',
              min=0, max=2, step=0.01)
setup_sliders('px0', 'px1', 'py0', 'py1', deschook=lambda x: x + " / π",
              min=-1, max=1, step=0.01)

ui = widgets.HBox([
    widgets.VBox([widgs['fx0'], widgs['fx1'], widgs['fy0'], widgs['fy1']]),
    widgets.VBox([widgs['Ax0'], widgs['Ax1'], widgs['Ay0'], widgs['Ay1']]),
    widgets.VBox([widgs['px0'], widgs['px1'], widgs['py0'], widgs['py1']]),
])


def update(ch):
    with fw.batch_update():
        #widgs['fy0'].value = widgs['fx0'].value
        #widgs['fy1'].value = widgs['fx1'].value
        
        #widgs['Ay0'].value = widgs['Ax0'].value
        #widgs['Ay1'].value = widgs['Ax1'].value
        
        # modifying source table to save a state on reruns
        for k,v in widgs.items():
            t.loc[0, k] = v.value
        build_orbit(t.loc[0], tt, pos)
        fw.data[0].x = pos['x']
        fw.data[0].y = pos['y']
        
        fw.data[1].x = pos['x']
        fw.data[1].y = pos['z']
        
        fw.data[2].x = pos['x']
        fw.data[2].y = pos['z']
        
        PR, Px, Py = make_periodograms(pos['x'], pos['y'])
        fw.data[3].y = PR
        fw.data[4].y = Px
        fw.data[5].y = Py
        #fw.layout.shapes[0].x0 = fx + fy
        #fw.layout.shapes[0].x1 = fx + fy

for k,v in widgs.items():
    widgs[k].observe(update, names="value")

update("whatever")
display(ui,fw)

FigureWidget({
    'data': [{'line': {'color': 'darkblue', 'width': 3},
              'showlegend': False,
   …

In [640]:
from IPython.display import HTML

HTML('''<script>
 $('div.input').hide();
</script>
''')